In [1]:
# Build version alpha 0.03
# Author: Joseph O'Neill
#
# Tool usage:
#   - To build a deep NN
#   - To test the accuracy of the model in predicting timeseries multivariate classification
# 
# Data:
#   - EEG recordings from an EMOTIV EPOC+
#   - EEG recording and file conversion from edf to csv with EMOTIV PRO software
#   - EEG recordings in two sets of subvocalized words: hello, world
#
# Neural Network:
#   - Conv. RNN based on the paper:
#       Cascade and Parallel Conv. NN on EEG-based Intention Recogn for BCI
#         by Lina Yao et al.
#   - As of this build haven't decided on Cascade or Parallel
#   - I am leaning towards trying the Cascade CRNN but it is still early
#
# Current usage abilities:
#   - Loads in all the csv files with glob
#   - ! EACH MAP IS MAPPED TO ITS CORRESPONDING ORIGINAL FILE NAME !
#   - maps the files by word (hello vs world) into 2 dictionaries or file maps
#       > (hello_file_map, world_file_map)
#   - Converts the file maps from array into meshes and maps into mesh maps
#       > (hello_mesh_map, world_mesh_map)
#   - Takes the mesh mapping and uses the z-score standardization
#      as instructed in the paper -> working towards Cascade CRNN
#   - Saves Z standard meshes into a new Z_map
#       > (hello_z_map, world_z_map)
##########################
# Added in Version a0.04 #
##########################
#   - Functions to simplify z_score_converion function
#       > (mesh_mean(), mesh_sigma(), mesh_z())

In [2]:
# URL to emotiv label meanings:
# https://emotiv.gitbook.io/emotivpro/exported_data_files/edf_files

import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [3]:
# glob -- For Unix style reading patterns -- https://docs.python.org/3/library/glob.html
# glob needed for *.csv
hello_file_path = "/home/joeyoneill/Desktop/cofc_research/HelloWorld-EEG/HelloWorld_Data/hello/*.csv"
world_file_path = "/home/joeyoneill/Desktop/cofc_research/HelloWorld-EEG/HelloWorld_Data/world/*.csv"


helloFiles = glob.glob(hello_file_path)
worldFiles = glob.glob(world_file_path)

In [4]:
# Reads in all hello .csv files into file_map dictionary
def read_map_emotiv_csv(df, filename, file_map):
    
    # delete EEG.Interpolated and EEG.Counter
    # not needed and in the way of df slicing
    del df['EEG.Interpolated']
    del df['EEG.Counter']
    del df['Timestamp']
    
    # saves needed columns into the df
    # the 14 channel readings
    # gets rid of excess
    # tail gives us only the last 2000 ms (2 s)
    # last 2000 ms needed due to procedure data was recorded
    df = df[df.columns[0:14]].tail(255)
           
    # resets index making first index 0; needed for nomalized code
    # delete 'index' needed because 
    # reset_index() causes 'index' column of orig indexes to be made
    df = df.reset_index()
    del df['index']
    
    # Transfers data from pd to a numpy array
    # Saves numpy array in dictionary
    # Array mapped to its filename
    # Array Usage: df_as_arr[ts][iteration]
    # such that ts is an integer timestep (0-255) not the actual Timestamp (Ts)
    # iteration:
    # 0 -> AF3, 1 -> F7, 2 -> F3, 3 -> FC5
    # 4 -> T7, 5 -> P7, 6 -> O1, 7 -> O2, 8 -> P8
    # 9 -> T8, 10 -> FC6, 11 -> F4, 12 -> F8, 13 -> AF4
    file_map[filename[-7:]] = df.to_numpy()

In [5]:
# Reads in all hello .csv files into hello_file_map dictionary
hello_file_map = {}

for filename in helloFiles:
    # Read in the CSV File to the dataframe
    df = pd.read_csv(filename)
    
    # normalizes the df and maps values into hello_file_map
    read_map_emotiv_csv(df, filename, hello_file_map)

In [6]:
# Reads in all world .csv files into world_file_map dictionary
world_file_map = {}

for filename in worldFiles:
    # Read in the CSV File to the dataframe
    df = pd.read_csv(filename)
    
    # normalizes the df and maps values into hello_file_map
    read_map_emotiv_csv(df, filename, world_file_map)

In [7]:
# Function to convert the np.array version
# of the data into mesh matrices as noted in
# Cascade and Parallel Conv. NN on EEG-based Intention Recogn for BCI
# by Lina Yao et al.
'''
Mesh matrix (such that each channel is their volatage at that timestep t):
0   0   AF3 AF4 0   0
F7  0   F3  F4  0   F8
0   FC5 0   0   FC6 0
T7  0   0   0   0   T8
0   0   0   0   0   0 
P7  0   0   0   0   P8
0   O1  0   0   O2  0
'''
def convert_to_mesh_matrix(file_map, mesh_map):
    for key in file_map.keys():
        mesh_list = []
        for i in range(len(file_map[key])):
            # creates skeleton for the mesh matrix
            # 6 x 7 2D matrix padded with zeros for spatial recognization
            mesh = np.zeros((7, 6))
            
            # REMEMBER:
            # 0 -> AF3, 1 -> F7, 2 -> F3, 3 -> FC5
            # 4 -> T7, 5 -> P7, 6 -> O1, 7 -> O2, 8 -> P8
            # 9 -> T8, 10 -> FC6, 11 -> F4, 12 -> F8, 13 -> AF4
            
            # AF3
            mesh[0][2] = file_map[key][i][0]

            # AF4
            mesh[0][3] = file_map[key][i][13]

            # F7
            mesh[1][0] = file_map[key][i][1]

            # F3
            mesh[1][2] = file_map[key][i][2]

            # F4
            mesh[1][3] = file_map[key][i][11]

            # F8
            mesh[1][5] = file_map[key][i][12]

            # FC5
            mesh[2][1] = file_map[key][i][3]

            # FC6
            mesh[2][4] = file_map[key][i][10]

            # T7
            mesh[3][0] = file_map[key][i][4]

            # T8
            mesh[3][5] = file_map[key][i][9]

            # P7
            mesh[5][0] = file_map[key][i][5]

            # P8
            mesh[5][5] = file_map[key][i][8]

            # O1
            mesh[6][1] = file_map[key][i][6]

            # O2
            mesh[6][4] = file_map[key][i][7]

            # adds newly created mesh matrix to the list
            mesh_list.append(mesh)
        
        # maps the mesh list to its file name
        mesh_map[key] = mesh_list

In [8]:
# initialization of empty mesh mappings
hello_mesh_map = {}
world_mesh_map = {}

# Converting rt -> mt and saving it to the mesh_maps
convert_to_mesh_matrix(hello_file_map, hello_mesh_map)
convert_to_mesh_matrix(world_file_map, world_mesh_map)

In [9]:
# Function returns the mean of the non-zero values of a mesh
# Used to simplify z_score_conversion() function
def mesh_mean(mesh, N_channels):
    total = 0.0
    # Getting the mean value of the mesh    
    for i in range(len(mesh)):
        for j in range(len(mesh[i])):
            if mesh[i][j] != 0:
                total = total + mesh[i][j]
    return total / N_channels

In [10]:
# Function returns sigma value of the non-zero values of a mesh
# Used to simplify z_score_conversion() function
def mesh_sigma(mesh, mean, N_channels):
    # Getting the sigma value of the mesh
    total = 0.0
    for i in range(len(mesh)):
        for j in range(len(mesh[i])):
            if mesh[i][j] != 0:
                total = total + ((mesh[i][j] - mean)**2)
    return math.sqrt(total / N_channels)

In [11]:
# Function returns a z standardized mesh
# Used to simplify z_score_conversion() function
def mesh_z(mesh, mean, sigma):
    # Z score normalization of mesh
    z_mesh = np.zeros((7, 6))
    for i in range(len(mesh)):
        for j in range(len(mesh[i])):
            if mesh[i][j] != 0:
                z_val = (mesh[i][j] - mean) / sigma
                z_mesh[i][j] = z_val
    return z_mesh

In [12]:
# Function to convert from votage mesh mapping
# to Z value mesh mapping
#
# Takes in mesh map, and empty Z map
# returns a filled Z map of all converted file values
def z_score_conversion(mesh_map, z_map, N_channels):
    
    # For each file
    for key in mesh_map.keys():

        z_mesh_list = []
        
        # for each mesh of the current file 0 - 254
        for i in range(len(mesh_map[key])):
            
            # Getting the mean value of the mesh    
            mean = mesh_mean(mesh_map[key][i], N_channels)
            
            # Getting the sigma value of the mesh
            sigma = mesh_sigma(mesh_map[key][i], mean, N_channels)
            
            # Z score normalization of mesh
            z_mesh = mesh_z(mesh_map[key][i], mean, sigma)
            
            # adds the mesh to its respective list to map to filename
            z_mesh_list.append(z_mesh)
        
        # List mapped to filename
        z_map[key] = z_mesh_list

In [13]:
# initialize empty Z mesh maps
hello_z_map = {}
world_z_map = {}

# Number of EEG channels for mean purposes
N_channels = 14

# Converting voltage mesh -> z mesh
z_score_conversion(hello_mesh_map, hello_z_map, N_channels)
z_score_conversion(world_mesh_map, world_z_map, N_channels)

In [14]:
print("hello: " + str(len(hello_z_map)) + "\nworld: " + str(len(world_z_map)))

hello: 40
world: 41
